## ARXIV -->Research Paper


In [18]:
from langchain_community.tools import WikipediaQueryRun,ArxivQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

In [19]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

In [20]:
print(wikipedia.run("HUNTER X HUNTER"))
print("--------------------------------------------------")
print(arxiv.run("machine learning"))

Page: Hunter × Hunter
Summary: Hunter × Hunter (pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha's shōnen manga magazine Weekly Shōnen Jump since March 1998, although the manga has frequently gone on extended hiatuses since 2006. Its chapters have been collected in 38 tankōbon volumes as of September 2024. The story focuses on a young boy named Gon Freecss who discovers that his father, who left him at a young age, is actually a world-renowned Hunter, a licensed professional who specializes in fantastical pursuits such as locating rare or unidentified animal species, treasure hunting, surveying unexplored enclaves, or hunting down lawless individuals. Gon departs on a journey to become a Hunter and eventually find his father. Along the way, Gon meets various other Hunters and encounters the paranormal.
Hunter × Hunter was adapted into a 62-episode anime television series by Nippon Animation and direc

In [21]:
tools = [wikipedia,arxiv]

In [54]:
from langchain_community.document_loaders import WebBaseLoader
import faiss
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
loader = WebBaseLoader("https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.web_base.WebBaseLoader.html")
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
data = splitter.split_documents(docs)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
dimension = len(embeddings.embed_query("hello world"))

# 4. Create FAISS index
index = faiss.IndexFlatL2(dimension)

# 5. Create FAISS vector store
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore({}),
    index_to_docstore_id={},
)

# 6. Add documents to the store
vector_store.add_documents(data)

['b675334d-50a6-4056-bf3e-7c4b8760ba06',
 'd432baba-9a4a-4bb5-9289-a30fc57b1f82',
 '75b36987-187a-49f1-babd-4e24fc748aa4',
 '3287f84e-a686-4cee-85b6-78730c5b4a4e',
 '1f8b2f37-56da-4124-a189-bc750f44cd71',
 'daa3dde0-210f-4a8b-8ed5-640ffec5fbc1',
 '2d571c45-a9af-4eaa-a663-21a97077d59c',
 '8db8ad76-1381-4f07-a707-dfcedf796dd2',
 '14905194-3aa2-4a67-9369-7dfb9572e563',
 '8aad40cd-eb09-480d-9274-add34ffaa789',
 'e66dbc04-876d-4f7d-bff5-19fef89cc502',
 'd7f9189f-9e4f-4af7-b712-24debe4b72ec',
 '87aa9ee0-3e6e-4940-9072-244e3dbae2ef',
 '11442aa1-03e0-4ba2-a079-22ca0779f1fd',
 '8e247b16-ccd1-4a80-9a48-ef5d96bbcbe4',
 'a1b98dfa-b701-4847-8c44-e22c59098dca',
 '3d678adc-698f-437c-aa96-b8ed724955ed',
 '0f1106d7-3680-472b-94d2-19e5ac3cb32f',
 'ddfea6b5-e1c1-40d2-99cd-43347d97e27d',
 'c2e86608-5ab2-4019-a05c-9c168e756081',
 '9044467f-da83-4bfd-a12f-a84effdbee8f']

In [55]:
retriever = vector_store.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002432805A110>, search_kwargs={})

In [56]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever=retriever,name="LangchainDocs",description="Useful for when you need to find information about Langchain")
retriever_tool

Tool(name='LangchainDocs', description='Useful for when you need to find information about Langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002437C16BB50>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002432805A110>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000002437C169360>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002432805A110>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{

In [57]:
tools = [wikipedia,arxiv,retriever_tool]


In [58]:
print(tools)

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'e:\\LANGGRAPH_LANGCHAIN_LANGSMITH\\.venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=3, lang='en', load_all_available_meta=False, doc_content_chars_max=4000)), ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)), Tool(name='LangchainDocs', description='Useful for when you need to find information about Langchain', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000002437C16BB50>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS obje

In [59]:
import os
from dotenv import load_dotenv
load_dotenv()
from langsmith import Client
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
client = Client()
prompt = client.pull_prompt("hwchase17/openai-functions-agent", include_model=True)

In [60]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [61]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.3-70b-versatile")

In [62]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
agent = create_openai_tools_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools,verbose=True)

In [63]:
agent_executor.invoke({"input":"what is hunter x hunter about? also find me a research paper on machine learning and give me the link to the paper and also tell me what is Langchain?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Hunter x Hunter'}`


Page: Hunter × Hunter
Summary: Hunter × Hunter (pronounced "hunter hunter") is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha's shōnen manga magazine Weekly Shōnen Jump since March 1998, although the manga has frequently gone on extended hiatuses since 2006. Its chapters have been collected in 38 tankōbon volumes as of September 2024. The story focuses on a young boy named Gon Freecss who discovers that his father, who left him at a young age, is actually a world-renowned Hunter, a licensed professional who specializes in fantastical pursuits such as locating rare or unidentified animal species, treasure hunting, surveying unexplored enclaves, or hunting down lawless individuals. Gon departs on a journey to become a Hunter and eventually find his father. Along the way, Gon meets various other Hunters and encounters the paranormal

{'input': 'what is hunter x hunter about? also find me a research paper on machine learning and give me the link to the paper and also tell me what is Langchain?',
 'output': 'Here are the answers to your questions:\n\n1. Hunter x Hunter is a Japanese manga series written and illustrated by Yoshihiro Togashi. It has been serialized in Shueisha\'s shōnen manga magazine Weekly Shōnen Jump since March 1998. The story focuses on a young boy named Gon Freecss who discovers that his father, who left him at a young age, is actually a world-renowned Hunter, a licensed professional who specializes in fantastical pursuits such as locating rare or unidentified animal species, treasure hunting, surveying unexplored enclaves, or hunting down lawless individuals.\n\n2. Here are a few research papers on machine learning:\n* "Machine Learning Testing: Survey, Landscapes and Horizons" by Jie M. Zhang, Mark Harman, Lei Ma, and Yang Liu (2019)\n* "Human-in-the-loop Machine Learning: A Macro-Micro Perspec